In [29]:
import os
import numpy as np
import librosa
import tensorflow as tf
import gc
import pickle
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve

In [ ]:
NUM_CLASSES = 2  

X = np.load('../SavedFeatures/X_f0.npy')
y = np.load('../SavedFeatures/y_f0.npy')

y_encoded = to_categorical(y, NUM_CLASSES)
split_index = int(0.8 * len(X))
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y_encoded[:split_index], y_encoded[split_index:]

In [ ]:
del X,y
gc.collect()

In [24]:
def cus_CNN(input_shape, num_classes):
    model_input = Input(shape=input_shape) 
    
    x = Conv1D(32, kernel_size=3, activation='relu')(model_input)
    x = BatchNormalization()(x) 
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.4)(x)  
    
    x = Conv1D(64, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)  

    x = Conv1D(128, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.2)(x)  
    x = Flatten()(x)

    x = Dropout(0.2)(x)  
    output = Dense(num_classes, activation='softmax')(x) 
    
    
    return Model(inputs=model_input, outputs=output)

In [25]:
cnnmodel = cus_CNN(input_shape=(109, 1), num_classes=2)

cnnmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
tf.keras.backend.clear_session()

In [26]:
cnnmodel.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10


796/796 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6243 - loss: 0.7590 - val_accuracy: 0.7362 - val_loss: 0.6012
Epoch 2/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 1s 993us/step - accuracy: 0.7163 - loss: 0.6217 - val_accuracy: 0.7462 - val_loss: 0.5811
Epoch 3/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 1s 974us/step - accuracy: 0.7387 - loss: 0.5844 - val_accuracy: 0.7481 - val_loss: 0.5759
Epoch 4/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 1s 977us/step - accuracy: 0.7361 - loss: 0.5804 - val_accuracy: 0.7563 - val_loss: 0.5601
Epoch 5/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 1s 974us/step - accuracy: 0.7407 - loss: 0.5658 - val_accuracy: 0.7557 - val_loss: 0.5416
Epoch 6/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 1s 960us/step - accuracy: 0.7477 - loss: 0.5537 - val_accuracy: 0.7619 - val_loss: 0.5354
Epoch 7/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 1s 949us/step - accuracy: 0.7577 - loss: 0.5399 - val_accuracy: 0.7657 - val_loss: 0.5224
Epoch 8/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7600 - loss: 0.5337 - val_accuracy: 0

#### Evaluation Metrics

In [ ]:
test_loss, test_accuracy = cnnmodel.evaluate(X_val, y_val, verbose=0)
y_pred = cnnmodel.predict(X_val)
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_val.argmax(axis=1)


f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')


# Tính EER
eers = []
print("===================CNN - F0===================")
print(f"Test Accuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")


for i in range(y_pred.shape[1]):
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]
    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")


mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
===================CNN - F0===================
Test Accuracy: 0.77450
F1-Score: 0.77068
Class 0: EER = 0.24463 at threshold 0.69705
Class 1: EER = 0.24468 at threshold 0.30305
EER: 0.24466


In [ ]:
# Save model as H5 File
cnnmodel.save("../SavedModel/F0_cus_cnn.h5")

In [30]:

del cnnmodel, X_train, X_val, y_train, y_val  
gc.collect()


1369